In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template, url_for
import joblib
import pandas as pd
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from datetime import timedelta  
import datetime
from sklearn.preprocessing import LabelEncoder

from numpy import mean
from numpy import std

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Define class function
class MultiColumnLabelEncoder:

    def __init__(self, columns=None):
        self.columns = columns # array of column names to encode

    def fit(self, X, y=None):
        self.encoders = {}
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            self.encoders[col] = LabelEncoder().fit(X[col])
        return self

    def transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].transform(X[col])
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)

    def inverse_transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].inverse_transform(X[col])
        return output
    
# feature engineering for previous month average, maximum and minimum
def mth_avg (n):
    mth_avg = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].mean().reset_index()
    mth_avg = mth_avg.rename(columns={'total_qty':'mth%s_avg_qty' %n})
    mth_avg['Month'] = mth_avg['Month']+n
    mth_avg.loc[mth_avg.Month == (12+n), 'Year'] = mth_avg.loc[mth_avg.Month == (12 + n)].Year + 1 
    mth_avg['Month'] = mth_avg['Month'].replace((12+n), n)
    mth_avg['mth%s_key' %n] = mth_avg['Year'].map(str) + mth_avg['Month'].map(str) + mth_avg['distributor_name'].map(str) + mth_avg['customer_name'].map(str) + mth_avg['product_name'].map(str)
    mth_avg = mth_avg.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)
    return mth_avg

def mth_max(n):
    mth_max = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].max().reset_index()
    mth_max = mth_max.rename(columns={'total_qty':'mth%s_max_qty' %n})
    mth_max['Month'] = mth_max['Month']+n
    mth_max.loc[mth_max.Month == (12+n), 'Year'] = mth_max.loc[mth_max.Month == (12+n)].Year + 1 
    mth_max['Month'] = mth_max['Month'].replace((12+n), n)
    mth_max['mth%s_key' %n] = mth_max['Year'].map(str) + mth_max['Month'].map(str) + mth_max['distributor_name'].map(str) + mth_max['customer_name'].map(str) + mth_max['product_name'].map(str)
    mth_max = mth_max.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)
    return mth_max

def mth_min(n):
    mth_min = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].min().reset_index()
    mth_min = mth_min.rename(columns={'total_qty':'mth%s_min_qty' %n})
    mth_min['Month'] = mth_min['Month']+n
    mth_min.loc[mth_min.Month == (12+n), 'Year'] = mth_min.loc[mth_min.Month == (12+n)].Year + 1 
    mth_min['Month'] = mth_min['Month'].replace((12+n), n)
    mth_min['mth%s_key' %n] = mth_min['Year'].map(str) + mth_min['Month'].map(str) + mth_min['distributor_name'].map(str) + mth_min['customer_name'].map(str) + mth_min['product_name'].map(str)
    mth_min = mth_min.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)
    return mth_min

# merge into one dataframe
def mth(n):
    global df
    mth = pd.merge(mth_avg(n), mth_max(n), how='left', on=('mth%s_key'%n))
    mth = pd.merge(mth, mth_min(n), how='left', on=('mth%s_key'%n))
    df['mth%s_key' %n] = df['Year'].map(str) + df['Month'].map(str) + df['distributor_name'].map(str) + df['customer_name'].map(str) + df['product_name'].map(str)
    # merge with df
    df = pd.merge(df, mth, how='left', on=('mth%s_key' %n))
    df['mth%s_avg_qty'%n] = df['mth%s_avg_qty'%n].fillna(0)
    df['mth%s_max_qty'%n] = df['mth%s_max_qty'%n].fillna(0)
    df['mth%s_min_qty'%n] = df['mth%s_min_qty'%n].fillna(0)
    return df

# feature engineering for last year month average, maximum and minimum
def last_yr_mth_avg(n):
    last_yr_mth_avg = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].mean().reset_index()
    last_yr_mth_avg = last_yr_mth_avg.rename(columns={'total_qty':'last_yr%s_mth_avg_qty'%n})
    last_yr_mth_avg['Year'] = last_yr_mth_avg['Year']+n
    last_yr_mth_avg['last_yr%s_mth_key'%n] = last_yr_mth_avg['Year'].map(str) + last_yr_mth_avg['Month'].map(str) + last_yr_mth_avg['distributor_name'].map(str) + last_yr_mth_avg['customer_name'].map(str) + last_yr_mth_avg['product_name'].map(str)
    last_yr_mth_avg = last_yr_mth_avg.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)
    return last_yr_mth_avg

def last_yr_mth_max(n):
    last_yr_mth_max = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].max().reset_index()
    last_yr_mth_max = last_yr_mth_max.rename(columns={'total_qty':'last_yr%s_mth_max_qty'%n})
    last_yr_mth_max['Year'] = last_yr_mth_max['Year']+n
    last_yr_mth_max['last_yr%s_mth_key'%n] = last_yr_mth_max['Year'].map(str) + last_yr_mth_max['Month'].map(str) + last_yr_mth_max['distributor_name'].map(str) + last_yr_mth_max['customer_name'].map(str) + last_yr_mth_max['product_name'].map(str)
    last_yr_mth_max = last_yr_mth_max.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)   
    return last_yr_mth_max

def last_yr_mth_min(n):
    last_yr_mth_min = df.groupby(['Year','Month','distributor_name','customer_name','product_name'])['total_qty'].min().reset_index()
    last_yr_mth_min = last_yr_mth_min.rename(columns={'total_qty':'last_yr%s_mth_min_qty'%n})
    last_yr_mth_min['Year'] = last_yr_mth_min['Year']+n
    last_yr_mth_min['last_yr%s_mth_key'%n] = last_yr_mth_min['Year'].map(str) + last_yr_mth_min['Month'].map(str) + last_yr_mth_min['distributor_name'].map(str) + last_yr_mth_min['customer_name'].map(str) + last_yr_mth_min['product_name'].map(str)
    last_yr_mth_min = last_yr_mth_min.drop(['Year','Month','distributor_name','customer_name','product_name'], axis=1)
    return last_yr_mth_min

def last_yr_mth(n):
    # merge into one dataframe
    global df
    last_yr_mth = pd.merge(last_yr_mth_avg(n), last_yr_mth_max(n), how='left', on=('last_yr%s_mth_key'%n))
    last_yr_mth = pd.merge(last_yr_mth, last_yr_mth_min(n), how='left', on=('last_yr%s_mth_key'%n))
    df['last_yr%s_mth_key'%n] = df['Year'].map(str) + df['Month'].map(str) + df['distributor_name'].map(str) + df['customer_name'].map(str) + df['product_name'].map(str)
    # merge with df
    df = pd.merge(df, last_yr_mth, how='left', on=('last_yr%s_mth_key'%n))
    df['last_yr%s_mth_avg_qty'%n] = df['last_yr%s_mth_avg_qty'%n].fillna(0)
    df['last_yr%s_mth_max_qty'%n] = df['last_yr%s_mth_max_qty'%n].fillna(0)
    df['last_yr%s_mth_min_qty'%n] = df['last_yr%s_mth_min_qty'%n].fillna(0)
    return df

#week
def week_avg(n):
    week_avg = df.groupby(['ww','distributor_name','customer_name','product_name'])['total_qty'].mean().reset_index()
    week_avg = week_avg.rename(columns={'total_qty':'week%s_avg_qty' %n})
    week_avg['week%s_key'%n] = week_avg['ww'].map(str) + week_avg['distributor_name'].map(str) + week_avg['customer_name'].map(str) + week_avg['product_name'].map(str)
    week_avg = week_avg.drop(['ww','distributor_name','customer_name','product_name'], axis=1)
    return week_avg

def week_max(n):
    week_max = df.groupby(['ww','distributor_name','customer_name','product_name'])['total_qty'].max().reset_index()
    week_max = week_max.rename(columns={'total_qty':'week%s_max_qty' %n})
    week_max['week%s_key'%n] = week_max['ww'].map(str) + week_max['distributor_name'].map(str) + week_max['customer_name'].map(str) + week_max['product_name'].map(str)
    week_max = week_max.drop(['ww','distributor_name','customer_name','product_name'], axis=1)
    return week_max

def week_min(n):  
    week_min = df.groupby(['ww','distributor_name','customer_name','product_name'])['total_qty'].min().reset_index()
    week_min = week_min.rename(columns={'total_qty':'week%s_min_qty' %n})
    week_min['week%s_key'%n] = week_min['ww'].map(str) + week_min['distributor_name'].map(str) + week_min['customer_name'].map(str) + week_min['product_name'].map(str)
    week_min = week_min.drop(['ww','distributor_name','customer_name','product_name'], axis=1)
    return week_min

def week(n):
    # merge into one dataframe
    global df
    week = pd.merge(week_avg(n), week_max(n), how='left', on=('week%s_key'%n))
    week = pd.merge(week, week_min(n), how='left', on=('week%s_key'%n))
    df['week%s'%n] = (df['txn_date']-timedelta(weeks = n)).dt.strftime('%G%V').astype(int)
    df['week%s_key'%n] = df['week%s'%n].map(str) + df['distributor_name'].map(str) + df['customer_name'].map(str) + df['product_name'].map(str)
    # merge with df
    df = pd.merge(df, week, how='left', on=('week%s_key'%n))
    df['week%s_avg_qty'%n] = df['week%s_avg_qty'%n].fillna(0)
    df['week%s_max_qty'%n] = df['week%s_max_qty'%n].fillna(0)
    df['week%s_min_qty'%n] = df['week%s_min_qty'%n].fillna(0)
    return df

#weeklag
def weeklag(n):
    # feature engineering for previous week lag 
    global df
    weeklag = df.groupby(['ww','dayofweek','distributor_name','customer_name','product_name'])['total_qty'].sum().reset_index()
    weeklag = weeklag.rename(columns={'total_qty':'weeklag%s_qty'%n})
    weeklag['weeklag%s_key'%n] = weeklag['ww'].map(str) + weeklag['dayofweek'].map(str) + weeklag['distributor_name'].map(str) + weeklag['customer_name'].map(str) + weeklag['product_name'].map(str)
    # drop extra columns
    weeklag = weeklag.drop(['ww','dayofweek','distributor_name','customer_name','product_name'], axis=1)
    df['weeklag%s_key'%n] = df['week%s'%n].map(str) + df['dayofweek'].map(str) + df['distributor_name'].map(str) + df['customer_name'].map(str) + df['product_name'].map(str)
    # merge with df
    df = pd.merge(df, weeklag, how='left', on=('weeklag%s_key'%n))
    df['weeklag%s_qty'%n] = df['weeklag%s_qty'%n].fillna(0)
    return df

# feature engineering for previous year1 average, maximum and minimum
def year_avg(n):
    year_avg = df.groupby(['Year','distributor_name','customer_name','product_name'])['total_qty'].mean().reset_index()
    year_avg = year_avg.rename(columns={'total_qty':'year%s_avg_qty' %n})
    year_avg['Year'] = year_avg['Year']+n
    year_avg['year%s_key'%n] = year_avg['Year'].map(str) + year_avg['distributor_name'].map(str) + year_avg['customer_name'].map(str) + year_avg['product_name'].map(str)
    year_avg = year_avg.drop(['Year','distributor_name','customer_name','product_name'], axis=1)
    return year_avg

def year_max(n):
    year_max = df.groupby(['Year','distributor_name','customer_name','product_name'])['total_qty'].max().reset_index()
    year_max = year_max.rename(columns={'total_qty':'year%s_max_qty' %n})
    year_max['Year'] = year_max['Year']+n
    year_max['year%s_key'%n] = year_max['Year'].map(str) + year_max['distributor_name'].map(str) + year_max['customer_name'].map(str) + year_max['product_name'].map(str)
    year_max = year_max.drop(['Year','distributor_name','customer_name','product_name'], axis=1)
    return year_max

def year_min(n):
    year_min = df.groupby(['Year','distributor_name','customer_name','product_name'])['total_qty'].min().reset_index()
    year_min = year_min.rename(columns={'total_qty':'year%s_min_qty' %n})
    year_min['Year'] = year_min['Year']+n
    year_min['year%s_key'%n] = year_min['Year'].map(str) + year_min['distributor_name'].map(str) + year_min['customer_name'].map(str) + year_min['product_name'].map(str)
    year_min = year_min.drop(['Year','distributor_name','customer_name','product_name'], axis=1)
    return year_min

def year(n):
    # merge into one dataframe
    global df
    year = pd.merge(year_avg(n), year_max(n), how='left', on=('year%s_key'%n))
    year = pd.merge(year, year_min(n), how='left', on=('year%s_key'%n))
    df['year%s_key'%n] = df['Year'].map(str) + df['distributor_name'].map(str) + df['customer_name'].map(str) + df['product_name'].map(str)
    # merge with df
    df = pd.merge(df, year, how='left', on=('year%s_key'%n))
    df['year%s_avg_qty'%n] = df['year%s_avg_qty'%n].fillna(0)
    df['year%s_max_qty'%n] = df['year%s_max_qty'%n].fillna(0)
    df['year%s_min_qty'%n] = df['year%s_min_qty'%n].fillna(0)
    return df

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# create flask application instance
app = Flask(__name__)

# Load the pipeline (MultiColumnLabelEncoder,model)
modelPipeline=joblib.load('modelPipeline.pkl')

# Read csv
df = pd.read_csv("pivot.csv", quotechar='"',parse_dates=['txn_date'])

#Convert function return value into an HTTP response to be displayed by main url ('/')
@app.route('/')
def home():
    return render_template('home.html')
    
#Convert function return value into an HTTP response to be displayed by url ('/predict') after receiving post request
@app.route('/predict',methods = ['POST'])
def predict():
    global df
    #Load data
    int_features = [str(x) for x in request.form.values()]
    int_features[0]=np.datetime64(int_features[0])
    int2_features = [np.array(int_features)]
    a=pd.DataFrame(int2_features,columns=['txn_date','distributor_name','customer_name','product_name'])
    df=pd.concat([df, a], ignore_index=True)
    
    #Feature generation
    df['ww'] = df['txn_date'].dt.strftime('%G%V').astype(int)
    df['Year'] = df['txn_date'].dt.year
    df['Month'] = df['txn_date'].dt.month
    df['dayofweek'] = df['txn_date'].dt.dayofweek
    
    for x in range(1, 4):
        df = mth(x)
        df = last_yr_mth(x)
        df = week(x)
        df = weeklag(x)
        df = year(x)   

    # drop extra columns
    df = df.drop(['txn_date','mth1_key','mth2_key','mth3_key','last_yr1_mth_key','last_yr2_mth_key','last_yr3_mth_key','week1','week2','week3','week1_key','week2_key','week3_key','weeklag1_key','weeklag2_key','weeklag3_key','year1_key','year2_key','year3_key'], axis=1)
    
   
    categorical_variables = list(df.select_dtypes(['object']).columns)
    for var in categorical_variables:
        df[var] = df[var].astype(str)
    
    # split data
    test = df.tail(len(a))
    X_test = test.drop(['total_qty'],1)
    y_test = test.total_qty
    
    # Predict 
    y_pred = modelPipeline.predict(X_test)
    
    # Sum up predicted sales in different levels 
    test['pred']=y_pred
    m = test.groupby(['Year','Month','distributor_name','customer_name','product_name'])['pred'].sum().reset_index()
    m_monthly = test.groupby(['Year','Month'])['total_qty','pred'].sum().reset_index()
    m_distributor = test.groupby(['Year','Month','distributor_name'])['pred'].sum().reset_index()
    m_distributor_customer = test.groupby(['Year','Month','distributor_name','customer_name'])['pred'].sum().reset_index()
    m_product = test.groupby(['Year','Month','product_name'])['pred'].sum().reset_index()
    
    # check the prediction value using console
    print(m.pred[0])
    print(m_monthly.pred[0])
    print(m_distributor.pred[0])
    print(m_distributor_customer.pred[0])
    print(m_product.pred[0])
    print(X_test)
    
    return render_template('result.html', prediction_text="Monthly Sales Prediction is {:.2f}".format(m_monthly.pred[0]),
                           prediction_text1="Monthly distributor Sales Prediction is {:.2f}".format(m_distributor.pred[0]),
                           prediction_text2="Monthly customer Sales Prediction is {:.2f}".format(m_distributor_customer.pred[0]),
                           prediction_text3="Monthly Product Sales Prediction is {:.2f}".format (m_product.pred[0]))

#run the app
if __name__ == '__main__':
    #app.run(debug=True)
    app.run(threaded=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Dec/2020 17:58:40] "GET / HTTP/1.1" 200 -
